In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

df_team_names = pd.read_csv('inputs/all_sources_team_names.csv')
team_names = df_team_names['fanfooty_team_name_long'].tolist()
url = "http://www.fanfooty.com.au/aflteams/"
df_player_list = pd.DataFrame()

for team in team_names:
    row_list = []
    column_headers = []
    res = requests.get("{}{}/".format(url, team))
    soup = BeautifulSoup(res.text, features='lxml')
    table = soup.select("#content-area > table:nth-child(10)")
    table = table[0]
    th = table.findAll(lambda tag: tag.name=='th')
    for header in th:
        column_headers.append(header.text)
    rows = table.findAll(lambda tag: tag.name=='tr')
    for tr in rows[1:]:
        td = tr.find_all('td')
        row = [tr.text for tr in td]
        row_list.append(row)
    df = pd.DataFrame(row_list, columns=column_headers)
    df['team'] = team
    df_player_list = pd.concat([df_player_list, df])
    
df_player_list['short_team'] = df_player_list['team'].map(dict(df_team_names[['fanfooty_team_name_long', 'sc_team_name']].values))
df_player_list

,Name,Type,#,DOB,Age,Height,Weight,Gm,SOO,Recruited from,team,short_team
0,Rory Atkins,Mid-Med,21,"12 Jul, 1994",25 y 229 d,186 cm,85 kg,97,VIC,Calder Cannons,adelaidecrows,ADE
1,Luke Brown,Def-Med-Gen,16,"22 Sep, 1992",27 y 157 d,181 cm,81 kg,150,SA,Norwood,adelaidecrows,ADE
2,Brad Crouch,Mid-Med,2,"14 Jan, 1994",26 y 43 d,186 cm,85 kg,83,VIC,North Ballarat Rebels,adelaidecrows,ADE
3,Matt Crouch,Mid-Med,5,"21 Apr, 1995",24 y 311 d,182 cm,80 kg,109,VIC,North Ballarat Rebels,adelaidecrows,ADE
4,Ben Davis,Fwd-Med-Gen,40,"19 May, 1997",22 y 283 d,187 cm,85 kg,1,NSW,UNSW,adelaidecrows,ADE
...,...,...,...,...,...,...,...,...,...,...,...,...
40,Callum M. Brown,Fwd-Med-Gen,46,"15 Aug, 2000",19 y 195 d,188 cm,91 kg,,N/A,County Derry,westernsydneygiants,GWS
41,Jack Buckley,Fwd-Tall-Key,44,"17 Dec, 1997",22 y 71 d,193 cm,87 kg,,NSW,UNSW,westernsydneygiants,GWS
42,Thomas Sheridan,,11,"28 Oct, 1993",26 y 121 d,187 cm,82 kg,81,VIC,Calder Cannons,westernsydneygiants,GWS
43,Zachary Sproule,Util-Tall,28,"12 May, 1998",21 y 290 d,197 cm,92 kg,1,NSW,Murray Bushrangers,westernsydneygiants,GWS


In [2]:
# df_player_list.to_csv('2020_fanfooty_player_list.csv')